In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  ##loading all environment variables


groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Shehjad I am chief AI engineer")])

In [ ]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Shehjad I am chief AI engineer"),
        AIMessage(content="Nice to meet you, Shehjad. As a chief AI engineer, you must be working on some exciting projects. What areas of AI are you currently focused on, or what kind of projects have you been working on recently?"),
        HumanMessage(content="Hey whats my name and what do i do?")
    ]
)

### message history

When we use MessageHistory, it helps the chatbot remember previous chats automatically.

Before this, we had to manually pass every message (Human + AI).

Now, the MessageHistory class does that for us — it stores all chats (inputs + outputs) automatically in memory.


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
response=with_message_history.invoke(
    [(HumanMessage(content="Hi , My name is Shehjad I am chief AI engineer"))],
    config=config,
)

In [ ]:
response.content

In [ ]:
response=with_message_history.invoke(
    [(HumanMessage(content="What's my name"))],
    config=config,
)
response

In [ ]:
##change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [(HumanMessage(content="What's my name"))],
    config=config1,
)
response.content

as u can see above it dont remember information when we change session id


In [ ]:
response=with_message_history.invoke(
    [(HumanMessage(content="my name is lala"))],
    config=config1,
)
response.content

In [ ]:
response=with_message_history.invoke(
    [(HumanMessage(content="does my name start with p.?"))],
    config=config1,
)
response.content

Prompt template

using prompt with history

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.Answer all the question to the best of your ability"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [ ]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is shehjad")]})

In [ ]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat3"}}

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is shehjad")],
    config=config
)
response.content

In [ ]:
##adding more complexity

prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.Answer all the question to the best of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [ ]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Shehjad")],"language":"Hindi"})
response.content

In [ ]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [ ]:
config={"configurable":{"session_id":"chat4}"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi i m shehjad")],"language":"hindi"},
    config=config
)
response.content

In [ ]:

response=with_message_history.invoke(
    {"messages":[HumanMessage(content="which language u r talking and whats my name")],"language":"hindi"},
    config=config
)
response.content